In [1]:
from __future__ import  division, print_function, absolute_import
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mne
import warnings
from mne.io import RawArray
from mne.epochs import concatenate_epochs
from mne import create_info, find_events, Epochs
from mne.channels import read_custom_montage
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from glob import glob
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.signal import welch
from mne import pick_types
import os.path
import scipy.io as sio
from cnn_class import cnn
import time
from sklearn.metrics import classification_report, roc_auc_score, auc, roc_curve, f1_score
from RnnAttention.attention import attention
from scipy import interp
from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA
warnings.filterwarnings('ignore')
##########################
# create MNE readable file 
##########################
def creat_mne_raw_object(fname):
    """Create a mne raw instance from csv file"""
    # Read EEG file
    data = pd.read_csv(fname)
    
    # get chanel names
    ch_names = list(data.columns[1:])
    
    # read EEG standard montage from mne
    montage = 'standard_1005'

    # events file
    ev_fname = fname.replace('_data','_events')
    # read event file
    events = pd.read_csv(ev_fname)
    events_names = events.columns[1:]
    events_data = np.array(events[events_names]).T
    
    # concatenate event file and data
    data = np.concatenate((1e-6*np.array(data[ch_names]).T,events_data))        
    
    # define channel type, the first is EEG, the last 6 are stimulations
    ch_type = ['eeg']*len(ch_names) + ['stim']*6
    
    # create and populate MNE info structure
    ch_names.extend(events_names)
    info = create_info(ch_names,sfreq=500.0,ch_types=ch_type)
    # info.set_montage(montage)
    #info['filename'] = fname
    
    # create raw object 
    raw = RawArray(data,info,verbose=False)
    return raw


####################################################################
# Data preprocessing, band_pass, event related 1.5s + 1.5s featuring
####################################################################
def data_gen(subjects):    
    X_train=[]
    y_train=[]
    n = 0
    for i in range(len(subjects)):
        n+=1
        subject = subjects[i]
        epochs_tot = []
        epochs_rest_tot = []
        fnames =  glob('../EEG/train/subj%d_series*_data.csv' % (subject))

        y = []
        for i,fname in enumerate(fnames):
            # read data 
            raw = creat_mne_raw_object(fname)
            #raw.plot(block=True)
            # pick eeg signal
            picks = pick_types(raw.info,eeg=True)
            raw.filter(7,35, picks=picks, method='iir', n_jobs=-1, verbose=False)        
            # Filter data for alpha frequency and beta band
            ##########
            # Events #
            ##########
            ###############id = 'Replace'#################        
            # get event posision corresponding to Replace
            events = find_events(raw,stim_channel='Replace', verbose=False)
            # epochs signal for 1.5 second before the movement
            epochs = Epochs(raw, events, {'after' : 1}, 0.5, 2, proj=False, picks=picks, baseline=None, 
                            preload=True, verbose=False)
            epochs_tot.append(epochs)        
            # epochs signal for 1.5 second after the movement, this correspond to the rest period.
            epochs_rest = Epochs(raw, events, {'during' : 1}, -2, -0.5, proj=False, picks=picks, baseline=None, 
                                 preload=True, verbose=False)
            # Workaround to be able to concatenate epochs
            epochs_rest.shift_time(2.5, relative = True)
            epochs_rest_tot.append(epochs_rest)
            # adding "Replace" lable "5" 
            y.extend([5]*len(epochs_rest))

            #################### id = 'HandStart'####################
            # get event posision corresponding to HandStart
            events = find_events(raw,stim_channel='HandStart', verbose=False)
            # epochs signal for 1.5 second before the movement
            epochs = Epochs(raw, events, {'during' : 1}, 0.5, 2, proj=False, picks=picks, baseline=None, 
                            preload=True, verbose=False)       
            epochs_tot.append(epochs)        
            # epochs signal for 1.5 second after the movement, this correspond to the rest period.
            epochs_rest = Epochs(raw, events, {'before' : 1}, -2, -0.5, proj=False, picks=picks, baseline=None, 
                                 preload=True, verbose=False)

            # Workaround to be able to concatenate epochs
            epochs_rest.shift_time(2.5, relative = True)
            epochs_rest_tot.append(epochs_rest)
            # adding lable "1" of "HandStart"
            y.extend([1]*len(epochs_rest))
            '''
            #################### id = 'FirstDigitTouch'####################
            # get event posision corresponding to FirstDigitTouch
            events = find_events(raw,stim_channel='FirstDigitTouch', verbose=False)
            epochs = Epochs(raw, events, {'during' : 1}, 0.5, 2, proj=False, picks=picks, baseline=None, 
                            preload=True, verbose=False)       
            epochs_tot.append(epochs)        
            epochs_rest = Epochs(raw, events, {'before' : 1}, -2, -0.5, proj=False, picks=picks, baseline=None, 
                                 preload=True, verbose=False)
            epochs_rest.shift_time(2.5, relative = True)
            epochs_rest_tot.append(epochs_rest)
            # adding lable "2" of "FirstDigitTouch"
            y.extend([2]*len(epochs_rest))

            #################### id = 'BothStartLoadPhase'####################
            # get event posision corresponding to BothStartLoadPh - truncated to 15 characters
            events = find_events(raw,stim_channel='BothStartLoadPh', verbose=False)
            epochs = Epochs(raw, events, {'during' : 1}, 0.5, 2, proj=False, picks=picks, baseline=None, 
                            preload=True, verbose=False)       
            epochs_tot.append(epochs)        
            epochs_rest = Epochs(raw, events, {'before' : 1}, -2, -0.5, proj=False, picks=picks, baseline=None, 
                                 preload=True, verbose=False)        
            epochs_rest.shift_time(2.5, relative = True)
            epochs_rest_tot.append(epochs_rest)
            # adding lable "3" of "FirstDigitTouch"
            y.extend([3]*len(epochs_rest))
            
            
            #################### id = 'Liftoff'####################
            # get event posision corresponding to Liftoff 
            events = find_events(raw,stim_channel='LiftOff', verbose=False)
            epochs = Epochs(raw, events, {'during' : 1}, 0.5, 2, proj=False, picks=picks, baseline=None, 
                            preload=True, verbose=False)       
            epochs_tot.append(epochs)        
            epochs_rest = Epochs(raw, events, {'before' : 1}, -2, -0.5, proj=False, picks=picks, baseline=None, 
                                 preload=True, verbose=False)        
            epochs_rest.shift_time(2.5, relative = True)
            epochs_rest_tot.append(epochs_rest)
            # adding lable "4" of "FirstDigitTouch"
            y.extend([4]*len(epochs_rest))

            #################### id = 'BothReleased'####################
            # get event posision corresponding to BothReleased 
            events = find_events(raw,stim_channel='BothReleased', verbose=False)
            epochs = Epochs(raw, events, {'during' : 1}, 0.5, 2, proj=False, picks=picks, baseline=None, 
                            preload=True, verbose=False)       
            epochs_tot.append(epochs)        
            epochs_rest = Epochs(raw, events, {'before' : 1}, -2, -0.5, proj=False, picks=picks, baseline=None, 
                                 preload=True, verbose=False)        
            epochs_rest.shift_time(2.5, relative = True)
            epochs_rest_tot.append(epochs_rest)
            # adding lable "6" of "BothReleased"
            y.extend([6]*len(epochs_rest))
            '''

        epochs_during = concatenate_epochs(epochs_tot)
        epochs_rests = concatenate_epochs(epochs_rest_tot)

        #get data 
        X_during = epochs_during.get_data()
        X_rests = epochs_rests.get_data()
        
        X = np.concatenate((X_during,X_rests),axis=1)
        
        pca = UnsupervisedSpatialFilter(PCA(22), average=False)
        X = pca.fit_transform(X)
        print ("'X' after PCA shape: ",X.shape)
        y = np.array(y)
        
        #y = np.array(y)
        print ("subject",subject,X.shape)

        if n == 1:
            X_train = X
            y_train = y
        else:
            X_train = np.append(X_train,X,axis =0)
            y_train = np.append(y_train,y,axis =0)

        print ("data stack shape:",X_train.shape,y_train.shape)

    # generate feature dataset for next process 
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    print ('return data shape: ',X_train.shape,y_train.shape)
    return (X_train, y_train)


/home/yaoxiaojian/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/yaoxiaojian/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/yaoxiaojian/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/yaoxiaojian/anaconda3/lib/python3.7/site-packages/tensorflow/

In [2]:
for j in range(1,13):
    train_subject = [k for k in range(1,13) if k != j]
    test_subject = [j]
    train_X,train_y = data_gen(train_subject)
    test_X,test_y = data_gen(test_subject)
    
    idx = list(range(len(train_y)))
    np.random.shuffle(idx)
    train_X = train_X[idx]
    train_y = train_y[idx]    
    
    idx = list(range(len(test_y)))
    np.random.shuffle(idx)
    test_X = test_X[idx]
    test_y = test_y[idx] 
    
    train_X=train_X*100000
    test_X=test_X*100000
    # sio.savemat('/home/yaoxiaojian/Desktop/kaggle/EEG/TACR_indenpendent_4-class/T-set_sub'+str(j)+'.mat', {"train_x": train_X, "train_y": train_y, "test_x": test_X, "test_y": test_y})
    
    '''
    # train_X preprocess
    X_inputs = np.transpose(train_X, [1, 0, 2])
    X_inputs=X_inputs.reshape((X_inputs.shape[0],(X_inputs.shape[1]*X_inputs.shape[2])))
    X_inputs = np.transpose(X_inputs, [1, 0])    
    
    print ('X_inputs',X_inputs.shape)
    # test_X preprocess
    X_inputs1 = np.transpose(test_X, [1, 0, 2])
    X_inputs1 = X_inputs1.reshape((X_inputs1.shape[0],(X_inputs1.shape[1]*X_inputs1.shape[2])))
    X_inputs1 = np.transpose(X_inputs1, [1, 0])
    print ('X_inputs1',X_inputs1.shape)

    # X_inputs and Y_targets are np.array, while X,Y are tf.tensor class

    learning_rate = 0.01
    training_epochs = 5
    batch_size = 751
    display_step = 1
    n_input = 64
    X = tf.placeholder("float", [None, n_input])

    n_hidden_1 = 88
    n_hidden_2 = 44
    n_hidden_3 = 22

    weights = {
        'encoder_h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1], )),
        'encoder_h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], )),
        'encoder_h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3], )),
        'decoder_h1': tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_2], )),
        'decoder_h2': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_1], )),
        'decoder_h3': tf.Variable(tf.truncated_normal([n_hidden_1, n_input], )),
    }
    biases = {
        'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'encoder_b3': tf.Variable(tf.random_normal([n_hidden_3])),
        'decoder_b1': tf.Variable(tf.random_normal([n_hidden_2])),
        'decoder_b2': tf.Variable(tf.random_normal([n_hidden_1])),
        'decoder_b3': tf.Variable(tf.random_normal([n_input])),
    }


    def encoder(x):
        layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                       biases['encoder_b1']))
        layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                       biases['encoder_b2']))
        layer_3 = tf.add(tf.matmul(layer_2, weights['encoder_h3']),
                         biases['encoder_b3'])

        return layer_3


    def decoder(x):
        layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                       biases['decoder_b1']))
        layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                       biases['decoder_b2']))
        layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
                                       biases['decoder_b3']))
        return layer_3

    # iteration
    encoder_op = encoder(X)
    decoder_op = decoder(encoder_op)

    y_pred = decoder_op
    y_true = X

    cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2)) # cost func. MSE
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    encoder_result = []
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        total_batch = int(X_inputs.shape[0] / batch_size)
        for epoch in range(training_epochs):
            for i in range(total_batch):
                offset = (i * batch_size) 
                batch_xs = X_inputs[offset:(offset + batch_size), :] 
                #batch_ys = Y_targets[offset:(offset + batch_size), :]
                print ('.',end = '')
                _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
            if epoch % display_step == 0:
                print()
                print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(c))
        print("Optimization Finished!")

        # build encoder_result array as feature vector
        train_X=sess.run(encoder_op, feed_dict={X: X_inputs})      
        train_X=train_X.reshape(int(train_X.shape[0]/751),751,22)
        train_X=np.transpose(train_X,[0,2,1])

        test_X=sess.run(encoder_op, feed_dict={X: X_inputs1})      
        test_X=test_X.reshape(int(test_X.shape[0]/751),751,22)
        test_X=np.transpose(test_X,[0,2,1])

        print (train_X.shape, test_X.shape, train_y.shape,test_y.shape)
        display (test_X)

    sess.close()
    '''
    test_y	= test_y.ravel()
    train_y = train_y.ravel()
    display (test_y)

    train_y = np.asarray(pd.get_dummies(train_y), dtype = np.int8)
    test_y = np.asarray(pd.get_dummies(test_y), dtype = np.int8)

    #############
    # Set window
    #############

    window_size = 100
    step = 50
    n_channel = 22

    def windows(data, size, step):
        start = 0
        while ((start+size) < data.shape[0]):
            yield int(start), int(start + size)
            start += step

    def segment_signal_without_transition(data, window_size, step):
        segments = []
        for (start, end) in windows(data, window_size, step):
            if(len(data[start:end]) == window_size):
                segments = segments + [data[start:end]]
        return np.array(segments)

    def segment_dataset(X, window_size, step):
        win_x = []
        for i in range(X.shape[0]):
            win_x = win_x + [segment_signal_without_transition(X[i], window_size, step)]
        win_x = np.array(win_x)
        return win_x

    train_raw_x = np.transpose(train_X, [0, 2, 1])
    test_raw_x = np.transpose(test_X, [0, 2, 1])


    train_win_x = segment_dataset(train_raw_x, window_size, step)
    print("train_win_x shape: ", train_win_x.shape)
    test_win_x = segment_dataset(test_raw_x, window_size, step)
    print("test_win_x shape: ", test_win_x.shape)

    # [trial, window, channel, time_length]
    train_win_x = np.transpose(train_win_x, [0, 1, 3, 2])
    print("train_win_x shape: ", train_win_x.shape)

    test_win_x = np.transpose(test_win_x, [0, 1, 3, 2])
    print("test_win_x shape: ", test_win_x.shape)


    # [trial, window, channel, time_length, 1]
    train_x = np.expand_dims(train_win_x, axis = 4)
    test_x = np.expand_dims(test_win_x, axis = 4)

    num_timestep = train_x.shape[1]
    ###########################################################################
    # set model parameters
    ###########################################################################
    # kernel parameter
    kernel_height_1st	= 22
    kernel_width_1st 	= 45

    kernel_stride		= 1

    conv_channel_num	= 40

    # pooling parameter
    pooling_height_1st 	= 1
    pooling_width_1st 	= 56

    pooling_stride_1st = 10

    # full connected parameter
    attention_size = 512
    n_hidden_state = 64

    ###########################################################################
    # set dataset parameters
    ###########################################################################
    # input channel
    input_channel_num = 1

    # input height 
    input_height = train_x.shape[2]

    # input width
    input_width = train_x.shape[3]

    # prediction class
    num_labels = 2
    ###########################################################################
    # set training parameters
    ###########################################################################
    # set learning rate
    learning_rate = 5e-4

    # set maximum traing epochs
    training_epochs = 120

    # set batch size
    batch_size = 10

    # set dropout probability
    dropout_prob = 0.5

    # set train batch number per epoch
    batch_num_per_epoch = train_x.shape[0]//batch_size

    # instance cnn class
    padding = 'VALID'

    cnn_2d = cnn(padding=padding)

    # input placeholder
    X = tf.placeholder(tf.float32, shape=[None, input_height, input_width, input_channel_num], name = 'X')
    Y = tf.placeholder(tf.float32, shape=[None, num_labels], name = 'Y')
    train_phase = tf.placeholder(tf.bool, name = 'train_phase')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    # first CNN layer
    conv_1 = cnn_2d.apply_conv2d(X, kernel_height_1st, kernel_width_1st, input_channel_num, conv_channel_num, kernel_stride, train_phase)
    print("conv 1 shape: ", conv_1.get_shape().as_list())
    pool_1 = cnn_2d.apply_max_pooling(conv_1, pooling_height_1st, pooling_width_1st, pooling_stride_1st)
    print("pool 1 shape: ", pool_1.get_shape().as_list())

    pool1_shape = pool_1.get_shape().as_list()
    pool1_flat = tf.reshape(pool_1, [-1, pool1_shape[1]*pool1_shape[2]*pool1_shape[3]])

    fc_drop = tf.nn.dropout(pool1_flat, keep_prob)	

    lstm_in = tf.reshape(fc_drop, [-1, num_timestep, pool1_shape[1]*pool1_shape[2]*pool1_shape[3]])

    ################
    # pipline design
    #################

    ########################## RNN ########################
    cells = []
    for _ in range(2):
        cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_state, forget_bias=1.0, state_is_tuple=True)
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
        cells.append(cell)
    lstm_cell = tf.contrib.rnn.MultiRNNCell(cells)

    init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)

    # output ==> [batch, step, n_hidden_state]
    rnn_op, states = tf.nn.dynamic_rnn(lstm_cell, lstm_in, initial_state=init_state, time_major=False)

    ########################## attention ########################
    with tf.name_scope('Attention_layer'):
        attention_op, alphas = attention(rnn_op, attention_size, time_major = False, return_alphas=True)

    attention_drop = tf.nn.dropout(attention_op, keep_prob)	

    ########################## readout ########################
    y_ = cnn_2d.apply_readout(attention_drop, rnn_op.shape[2].value, num_labels)

    # probability prediction 
    y_prob = tf.nn.softmax(y_, name = "y_prob")

    # class prediction 
    y_pred = tf.argmax(y_prob, 1, name = "y_pred")

    ########################## loss and optimizer ########################
    # cross entropy cost function
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=Y), name = 'loss')


    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
    # set training SGD optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    # get correctly predicted object
    correct_prediction = tf.equal(tf.argmax(tf.nn.softmax(y_), 1), tf.argmax(Y, 1))

    ########################## define accuracy ########################
    # calculate prediction accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = 'accuracy')

    #############
    # train test 
    #############
    def multiclass_roc_auc_score(y_true, y_score):
        assert y_true.shape == y_score.shape
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        n_classes = y_true.shape[1]
        # compute ROC curve and ROC area for each class
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        # compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # compute macro-average ROC curve and ROC area
        # First aggregate all false probtive rates
        all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
        # Then interpolate all ROC curves at this points
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(n_classes):
            mean_tpr += interp(all_fpr, fpr[i], tpr[i])
        # Finally average it and compute AUC
        mean_tpr /= n_classes
        fpr["macro"] = all_fpr
        tpr["macro"] = mean_tpr
        roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
        return roc_auc
    # run with gpu memory growth
    config = tf.ConfigProto()
    # config.gpu_options.allow_growth = True

    train_acc = []
    test_acc = []
    best_test_acc = []
    train_loss = []
    with tf.Session(config=config) as session:
        session.run(tf.global_variables_initializer())
        best_acc = 0
        for epoch in range(training_epochs):
            pred_test = np.array([])
            true_test = []
            prob_test = []
            ########################## training process ########################
            for b in range(batch_num_per_epoch):
                offset = (b * batch_size) % (train_y.shape[0] - batch_size) 
                batch_x = train_x[offset:(offset + batch_size), :, :, :, :]
                batch_x = batch_x.reshape([len(batch_x)*num_timestep, n_channel, window_size, 1])
                batch_y = train_y[offset:(offset + batch_size), :]
                _, c = session.run([optimizer, cost], feed_dict={X: batch_x, Y: batch_y, keep_prob: 1-dropout_prob, train_phase: True})
            # calculate train and test accuracy after each training epoch
            if(epoch%1 == 0):
                train_accuracy 	= np.zeros(shape=[0], dtype=float)
                test_accuracy	= np.zeros(shape=[0], dtype=float)
                train_l 		= np.zeros(shape=[0], dtype=float)
                test_l			= np.zeros(shape=[0], dtype=float)
                # calculate train accuracy after each training epoch
                for i in range(batch_num_per_epoch):
                    ########################## prepare training data ########################
                    offset = (i * batch_size) % (train_y.shape[0] - batch_size) 
                    train_batch_x = train_x[offset:(offset + batch_size), :, :, :]
                    train_batch_x = train_batch_x.reshape([len(train_batch_x)*num_timestep, n_channel, window_size, 1])
                    train_batch_y = train_y[offset:(offset + batch_size), :]

                    ########################## calculate training results ########################
                    train_a, train_c = session.run([accuracy, cost], feed_dict={X: train_batch_x, Y: train_batch_y, keep_prob: 1.0, train_phase: False})

                    train_l = np.append(train_l, train_c)
                    train_accuracy = np.append(train_accuracy, train_a)
                print("("+time.asctime(time.localtime(time.time()))+") Epoch: ", epoch+1, " Training Cost: ", np.mean(train_l), "Training Accuracy: ", np.mean(train_accuracy))
                train_acc = train_acc + [np.mean(train_accuracy)]
                train_loss = train_loss + [np.mean(train_l)]
                # calculate test accuracy after each training epoch
                for j in range(batch_num_per_epoch):
                    ########################## prepare test data ########################
                    offset = (j * batch_size) % (test_y.shape[0] - batch_size) 
                    test_batch_x = test_x[offset:(offset + batch_size), :, :, :]
                    test_batch_x = test_batch_x.reshape([len(test_batch_x)*num_timestep, n_channel, window_size, 1])
                    test_batch_y = test_y[offset:(offset + batch_size), :]

                    ########################## calculate test results ########################
                    test_a, test_c, prob_v, pred_v = session.run([accuracy, cost, y_prob, y_pred], feed_dict={X: test_batch_x, Y: test_batch_y, keep_prob: 1.0, train_phase: False})

                    test_accuracy = np.append(test_accuracy, test_a)
                    test_l = np.append(test_l, test_c)
                    pred_test = np.append(pred_test, pred_v)
                    true_test.append(test_batch_y)
                    prob_test.append(prob_v)
                if np.mean(test_accuracy) > best_acc :
                    best_acc = np.mean(test_accuracy)
                true_test = np.array(true_test).reshape([-1, num_labels])
                prob_test = np.array(prob_test).reshape([-1, num_labels])
                auc_roc_test = multiclass_roc_auc_score(y_true=true_test, y_score=prob_test)
                f1 = f1_score (y_true=np.argmax(true_test, axis = 1), y_pred=pred_test, average = 'macro')
                print("("+time.asctime(time.localtime(time.time()))+") Epoch: ", epoch+1, "Test Cost: ", np.mean(test_l), 
                                                                                          "Test Accuracy: ", np.mean(test_accuracy), 
                                                                                          "Test f1: ", f1, 
                                                                                          "Test AUC: ", auc_roc_test['macro'], "\n")


'X' after PCA shape:  (520, 22, 751)
subject 2 (520, 22, 751)
data stack shape: (520, 22, 751) (520,)
'X' after PCA shape:  (520, 22, 751)
subject 3 (520, 22, 751)
data stack shape: (1040, 22, 751) (1040,)
'X' after PCA shape:  (520, 22, 751)
subject 4 (520, 22, 751)
data stack shape: (1560, 22, 751) (1560,)
'X' after PCA shape:  (520, 22, 751)
subject 5 (520, 22, 751)
data stack shape: (2080, 22, 751) (2080,)
'X' after PCA shape:  (520, 22, 751)
subject 6 (520, 22, 751)
data stack shape: (2600, 22, 751) (2600,)
'X' after PCA shape:  (520, 22, 751)
subject 7 (520, 22, 751)
data stack shape: (3120, 22, 751) (3120,)
'X' after PCA shape:  (520, 22, 751)
subject 8 (520, 22, 751)
data stack shape: (3640, 22, 751) (3640,)
'X' after PCA shape:  (520, 22, 751)
subject 9 (520, 22, 751)
data stack shape: (4160, 22, 751) (4160,)
'X' after PCA shape:  (520, 22, 751)
subject 10 (520, 22, 751)
data stack shape: (4680, 22, 751) (4680,)
'X' after PCA shape:  (520, 22, 751)
subject 11 (520, 22, 751)
da

array([5, 5, 5, 5, 5, 1, 5, 1, 5, 5, 1, 1, 1, 5, 1, 5, 5, 1, 5, 1, 1, 1,
       5, 1, 1, 5, 1, 1, 5, 5, 5, 1, 5, 1, 5, 5, 1, 5, 1, 5, 5, 1, 1, 5,
       5, 1, 1, 5, 1, 5, 5, 1, 5, 5, 1, 1, 5, 1, 1, 5, 1, 1, 5, 1, 1, 1,
       1, 5, 1, 5, 5, 5, 1, 5, 1, 1, 5, 5, 1, 5, 5, 1, 5, 5, 5, 1, 5, 1,
       1, 1, 5, 1, 1, 5, 5, 1, 5, 5, 1, 5, 1, 5, 5, 5, 5, 5, 1, 5, 1, 1,
       1, 1, 5, 5, 1, 1, 1, 1, 5, 5, 5, 1, 1, 1, 5, 5, 5, 1, 5, 1, 5, 1,
       1, 1, 1, 5, 5, 5, 1, 1, 1, 5, 5, 5, 5, 1, 1, 1, 1, 5, 1, 5, 5, 5,
       5, 1, 5, 5, 1, 5, 1, 5, 1, 5, 1, 1, 5, 1, 1, 5, 1, 5, 1, 5, 1, 1,
       1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 1, 1, 1, 1, 1, 1,
       5, 1, 5, 1, 5, 1, 5, 5, 5, 1, 1, 1, 1, 5, 1, 5, 1, 1, 1, 1, 5, 5,
       5, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 5, 5, 1, 1, 1, 5, 5,
       1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 5, 5, 5, 1, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 1, 1, 1, 1, 1, 5, 5, 1, 1, 1, 5, 1, 1, 1, 1, 5, 5, 5, 1, 5, 1,
       1, 5, 1, 5, 5, 1, 1, 5, 1, 5, 1, 1, 1, 1, 5,

train_win_x shape:  (5720, 14, 100, 22)
test_win_x shape:  (520, 14, 100, 22)
train_win_x shape:  (5720, 14, 22, 100)
test_win_x shape:  (520, 14, 22, 100)

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
conv 1 shape:  [None, 1, 56, 40]

pool 1 shape:  [None, 1, 1, 40]
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LS

(Sat Dec  5 15:09:06 2020) Epoch:  1 Test Cost:  0.6968417924184066 Test Accuracy:  0.5122377698446487 Test f1:  0.36224300176906954 Test AUC:  0.5216921345290658 

(Sat Dec  5 15:09:23 2020) Epoch:  2  Training Cost:  0.5952839398092323 Training Accuracy:  0.7122377637591395
(Sat Dec  5 15:09:28 2020) Epoch:  2 Test Cost:  0.6846255915356683 Test Accuracy:  0.5568181868981231 Test f1:  0.5306345581109939 Test AUC:  0.5751575432897384 

(Sat Dec  5 15:09:45 2020) Epoch:  3  Training Cost:  0.562811551967284 Training Accuracy:  0.719055943868377
(Sat Dec  5 15:09:49 2020) Epoch:  3 Test Cost:  0.7063668873551843 Test Accuracy:  0.5555944101220661 Test f1:  0.4815233519596739 Test AUC:  0.5464440072074722 

(Sat Dec  5 15:10:07 2020) Epoch:  4  Training Cost:  0.539355765689503 Training Accuracy:  0.7246503511092046
(Sat Dec  5 15:10:11 2020) Epoch:  4 Test Cost:  0.7145532596778202 Test Accuracy:  0.5452797249923428 Test f1:  0.468089773177384 Test AUC:  0.5211197095621304 

(Sat Dec  5

(Sat Dec  5 15:19:53 2020) Epoch:  31  Training Cost:  0.28015189387321054 Training Accuracy:  0.900874118600692
(Sat Dec  5 15:19:58 2020) Epoch:  31 Test Cost:  0.756666843551439 Test Accuracy:  0.5664335706017234 Test f1:  0.5432759672287886 Test AUC:  0.6076755075366393 

(Sat Dec  5 15:20:15 2020) Epoch:  32  Training Cost:  0.3033362544057044 Training Accuracy:  0.8819929995424264
(Sat Dec  5 15:20:20 2020) Epoch:  32 Test Cost:  0.8071047446423477 Test Accuracy:  0.5687063002398798 Test f1:  0.532343586238323 Test AUC:  0.6093770493301108 

(Sat Dec  5 15:20:37 2020) Epoch:  33  Training Cost:  0.37016461191737987 Training Accuracy:  0.8132867099313469
(Sat Dec  5 15:20:41 2020) Epoch:  33 Test Cost:  0.9116590642533102 Test Accuracy:  0.5201048988041345 Test f1:  0.3988431020467797 Test AUC:  0.5982626828114395 

(Sat Dec  5 15:20:59 2020) Epoch:  34  Training Cost:  0.3740440914342245 Training Accuracy:  0.8134615340016105
(Sat Dec  5 15:21:03 2020) Epoch:  34 Test Cost:  0.89

KeyboardInterrupt: 